In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# zipfileをcsvにする
import zipfile
zf1 = zipfile.ZipFile('/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip')
print(zf1.namelist())

zf2 = zipfile.ZipFile('/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip')
print(zf2.namelist()) 

zf3 = zipfile.ZipFile('/kaggle/input/ghouls-goblins-and-ghosts-boo/sample_submission.csv.zip')
print(zf3.namelist()) 

In [ ]:
zf1.extractall()
zf2.extractall()
zf3.extractall()

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.describe

In [ ]:
# csv内のヘッダーを確認
import csv
csv_file = open("train.csv", "r", encoding="ms932", errors="", newline="" )
f = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
header = next(f)
print(header)

In [ ]:
# ダミー
# pd.get_dummies(train['type'])
pd.get_dummies(train['color'])
pd.get_dummies(test['color'])

training = pd.get_dummies(train, columns=['color'])
training
testing = pd.get_dummies(test, columns=['color'])
testing

In [ ]:
# ダミー導入後の相関
training.corr()

In [ ]:
# ニューラルネットワーク
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import joblib

In [ ]:
# paramate
hidden_layer_sizes=(100,)
activation = 'relu'
solver = 'adam'
batch_size = 'auto'
alpha = 0.0001
random_state = 0
max_iter = 10000
# early_stopping = True

# 説明変数
# x = ['bone_length', 'rotting_flesh', 'hair_length', 'has_soul','color_black','color_blood','color_blue','color_clear','color_green','color_white']
x = ['bone_length', 'rotting_flesh', 'hair_length', 'has_soul']

train_X = training[x]

# 目的変数
y1 = ['type']
train_y1 = training[y1]

# 学習
clf = MLPClassifier(
    hidden_layer_sizes=hidden_layer_sizes,
    activation=activation,
    solver=solver,
    batch_size=batch_size,
    alpha=alpha,
    random_state=random_state,
    max_iter=max_iter,
#     early_stopping = early_stopping
    )

clf.fit(train_X, train_y1)

SAVE_TRAINED_DATA_PATH = 'train1.learn'
# 学習結果を出力
joblib.dump(clf, SAVE_TRAINED_DATA_PATH)

# 学習済ファイルのロード
clf1 = joblib.load(SAVE_TRAINED_DATA_PATH)



# 検証用の説明変数
# test_X = test['bone_length', 'rotting_flesh', 'hair_length', 'has_soul', 'color_black','color_blood','color_blue','color_clear','color_green','color_white']
test_X1 = testing[x]

# 学習結果の検証
predict_y1 = clf1.predict_proba(test_X1)
predict_Y1 = clf1.predict(test_X1)
print(predict_Y1)

# 学習データの精度
print("学習データの精度: {:.3f}".format(clf.score(train_X, train_y1)))

# テスト結果の精度
print("テスト結果の精度: {:.3f}".format(clf.score(test_X1, predict_Y1)))

In [ ]:
# 提出用データ
sub = pd.read_csv('sample_submission.csv')
sub['type'] = list(predict_Y1)
sub.to_csv('sample_submission.csv', index=False)